# TFL Bike data Analysis

#### Exploratory data analysis as part of my MSc thesis, "Using machine learning to predict Transport for London bike sharing habits in the post COVID-19 era".

The following code for downloading the data has been adopted from [Markus Hauru's](https://github.com/mhauru) analysis, 'Predicting Boris Bike usage'.



In [1]:
# importing libraries

import os
import pickle
import requests
import zipfile
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels.api as sm
from sklearn import linear_model, svm, neighbors, tree
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
from pathlib import Path
from timeit import default_timer as timer
from IPython.display import set_matplotlib_formats
from urllib.parse import urlparse

try:
    import xlrd
except Exception as e:
    msg = (
        "Please install the package xlrd: `pip install --user xlrd`"
        "It's an optional requirement for pandas, and we'll be needing it."
    )
    print(msg)
    raise e

#test for commit

In [2]:
# For pretty and exportable matplotlib plots.
# If you are running this yourself and want interactivity,
# try `%matplotlib widget` instead.
set_matplotlib_formats("svg")
%matplotlib inline
# %matplotlib widget
# Set a consistent plotting style across the notebook using Seaborn.
sns.set_style("darkgrid")
sns.set_context("notebook")
# Make pandas cooperate with pyplot
pd.plotting.register_matplotlib_converters()


C:\Users\EMoses\AppData\Local\Temp\ipykernel_23180\99259732.py:4: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("svg")


1. Processing and cleaning the bike data
Before getting anywhere with it, we'll need to process the bike data quite a bit. The data comes in CSV files, each of which covers a period of time. Up first, we need to download the data from the TfL website. If you are running this code yourself, here's a script that does that. Be warned though, it's almost seven gigs of data. You can run it repeatedly, and it'll only download data that it doesn't have already.

In [3]:
bikefolder = "data/bikes"

In [4]:
def download_file(datafolder, url, verbosity=0):
    """Download the data from the given URL into the datafolder, unless it's
    already there. Return path to downloaded file.
    """
    datafolder = Path(datafolder)
    datafolder.mkdir(parents=True, exist_ok=True)

    a = urlparse(url)
    filename = Path(os.path.basename(a.path))
    filepath = datafolder / filename
    # Don't redownload if we already have this file.
    if filepath.exists():
        if verbosity > 1:
            print("Already have {}".format(filename))
    else:
        if verbosity > 0:
            print("Downloading {}".format(filename))
        rqst = requests.get(url)
        rqst.raise_for_status()
        with open(filepath, "wb") as f:
            f.write(rqst.content)
    return filepath


In [5]:
# Adjust whether to print progress reports of the downloads.
# verbosity=0 is silence, verbosity=1 reports only when actually doing things,
# verbosity>1 also reports when there's nothing to do.
verbosity = 1

# Most files are individual CSV files, listed in bike_data_urls.txt. Download
# them.
urlsfile = "data/bikes/bike_data_urls.txt"
with open(urlsfile, "r") as f:
    urls = f.read().splitlines()
# There are a few comments in the file, marked by lines starting with #.
# Filter them out.
urls = [u for u in urls if u[0] != "#"]
for url in urls:
    download_file(bikefolder, url, verbosity)

# The early years come in zips. Download and unzip them.
zipsfolder = Path("data/bikes/bikezips")
bikezipurls = [
    "https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2012.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2013.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/2015TripDatazip.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/2016TripDataZip.zip",
]
# A list of CSV files that are already there. Only unzip if some of the files
# in the zip aren't present already.
current_csvs = sorted(os.listdir(bikefolder))
for url in bikezipurls:
    zippath = download_file(zipsfolder, url, verbosity)
    with zipfile.ZipFile(zippath, "r") as z:
        namelist = z.namelist()
        has_been_extracted = any(name not in current_csvs for name in namelist)
        if has_been_extracted:
            if verbosity > 0:
                print("Unzipping {}".format(zippath))
            z.extractall(bikefolder)
        else:
            if verbosity > 1:
                print("{} has already been extracted.".format(zippath))

# Finally, there's an odd one out: One week's data comes in as an .xlsx.
# Download it and use pandas to convert it to csv.
xlsxurl = "https://cycling.data.tfl.gov.uk/usage-stats/49JourneyDataExtract15Mar2017-21Mar2017.xlsx"
xlsxfile = download_file(bikefolder, xlsxurl)
csvfile = xlsxfile.with_suffix(".csv")
if not csvfile.exists():
    if verbosity > 0:
        print("Converting .xlsx to .csv.")
    pd.read_excel(xlsxfile).to_csv(csvfile, date_format="%d/%m/%Y %H:%M:%S")
else:
    if verbosity > 1:
        print("Already have {}".format(csvfile))

The data we have now lists on each line of the CSV file a single bike trip, with starting point and time, end point and time, and things like bike ID number. Here's an example.

In [ ]:
example_file  = Path(bikefolder) / Path("47JourneyDataExtract01Mar2017-07Mar2017.csv")
pd.read_csv(example_file, encoding="ISO-8859-2").head()

In [ ]:
bikefolder

In [6]:
from glob import glob 

# using glob to list all the csv file in the bikefolder filepath
all_csv = glob(bikefolder+str('/*.csv'))
all_csv

['data/bikes\\01aJourneyDataExtract10Jan16-23Jan16.csv',
 'data/bikes\\01bJourneyDataExtract24Jan16-06Feb16.csv',
 'data/bikes\\02aJourneyDataExtract07Feb16-20Feb2016.csv',
 'data/bikes\\02bJourneyDataExtract21Feb16-05Mar2016.csv',
 'data/bikes\\03JourneyDataExtract06Mar2016-31Mar2016.csv',
 'data/bikes\\04JourneyDataExtract01Apr2016-30Apr2016.csv',
 'data/bikes\\05JourneyDataExtract01May2016-17May2016.csv',
 'data/bikes\\06JourneyDataExtract18May2016-24May2016.csv',
 'data/bikes\\07JourneyDataExtract25May2016-31May2016.csv',
 'data/bikes\\08JourneyDataExtract01Jun2016-07Jun2016.csv',
 'data/bikes\\09JourneyDataExtract08Jun2016-14Jun2016.csv',
 'data/bikes\\1. Journey Data Extract 01Jan-05Jan13.csv',
 'data/bikes\\1. Journey Data Extract 04Jan-31Jan 12.csv',
 'data/bikes\\1. Journey Data Extract 05Jan14-02Feb14.csv',
 'data/bikes\\10. Journey Data Extract 18Aug-13Sep13.csv',
 'data/bikes\\10. Journey Data Extract 21Aug-22 Aug12.csv',
 'data/bikes\\10a Journey Data Extract 20Sep15-03Oct

In [7]:
# creating a list of csv files that contain '2019'
csv_2019 = [item for item in all_csv if '2019' in item]

csv_2019

['data/bikes\\142JourneyDataExtract26Dec2018-01Jan2019.csv',
 'data/bikes\\143JourneyDataExtract02Jan2019-08Jan2019.csv',
 'data/bikes\\144JourneyDataExtract09Jan2019-15Jan2019.csv',
 'data/bikes\\145JourneyDataExtract16Jan2019-22Jan2019.csv',
 'data/bikes\\146JourneyDataExtract23Jan2019-29Jan2019.csv',
 'data/bikes\\147JourneyDataExtract30Jan2019-05Feb2019.csv',
 'data/bikes\\148JourneyDataExtract06Feb2019-12Feb2019.csv',
 'data/bikes\\149JourneyDataExtract13Feb2019-19Feb2019.csv',
 'data/bikes\\150JourneyDataExtract20Feb2019-26Feb2019.csv',
 'data/bikes\\151JourneyDataExtract27Feb2019-05Mar2019.csv',
 'data/bikes\\152JourneyDataExtract06Mar2019-12Mar2019.csv',
 'data/bikes\\153JourneyDataExtract13Mar2019-19Mar2019.csv',
 'data/bikes\\154JourneyDataExtract20Mar2019-26Mar2019.csv',
 'data/bikes\\155JourneyDataExtract27Mar2019-02Apr2019.csv',
 'data/bikes\\156JourneyDataExtract03Apr2019-09Apr2019.csv',
 'data/bikes\\157JourneyDataExtract10Apr2019-16Apr2019.csv',
 'data/bikes\\158Journey

In [33]:
# using list comprehension that reads each csv file from the list and gnerators a sequence of dataframes
dfs = (pd.read_csv(csv) for csv in csv_2019)

# concatenate csvs them into a single DataFrame using pd.concat()
# ignore_index=True parameter resets the index of the resulting DataFrame, so that it is a continuous sequence of integers.
data_2019 = pd.concat(dfs, ignore_index=True)

In [12]:
print(data_2019.shape)
data_2019.head()

(10388411, 9)


,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,83252102,720,2077,31/12/2018 19:05,272,"Baylis Road, Waterloo",31/12/2018 18:53,94,"Bricklayers Arms, Borough"
1,83195883,120,10781,27/12/2018 19:47,93,"Cloudesley Road, Angel",27/12/2018 19:45,339,"Risinghill Street, Angel"
2,83196070,120,2977,27/12/2018 20:11,339,"Risinghill Street, Angel",27/12/2018 20:09,234,"Liverpool Road (N1 Centre), Angel"
3,83197932,660,10802,28/12/2018 07:35,282,"Royal London Hospital, Whitechapel",28/12/2018 07:24,698,"Shoreditch Court, Haggerston"
4,83176351,1380,15749,26/12/2018 11:55,785,"Aquatic Centre, Queen Elizabeth Olympic Park",26/12/2018 11:32,783,"Monier Road, Hackney Wick"


In [20]:
## Add some extra variables to the dataset for use later in filtering

import datetime

## Feeding a specififed date format speeds up the pd.to_datetime function immeasurably, especially over large datasets
## e.g. http://stackoverflow.com/questions/32034689/why-is-pandas-to-datetime-slow-for-non-standard-time-format-such-as-2014-12-31

format = "%d/%m/%Y %H:%M"

## Some routes had dates with a seconds component, whereas some didn't - the below code cuts these seconds off
data_2019['Start Date']= data_2019['Start Date'].str[:16]

data_2019['Start Date Converted']= pd.to_datetime(data_2019['Start Date'], format=format).dt.date

data_2019['Hours']= pd.to_datetime(data_2019['Start Date'], format=format).dt.hour

data_2019['Day']= pd.to_datetime(data_2019['Start Date'], format=format).dt.weekday

data_2019.head()


,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,Start Date Converted,Hours,Day
0,83252102,720,2077,31/12/2018 19:05,272,"Baylis Road, Waterloo",31/12/2018 18:53,94,"Bricklayers Arms, Borough",2018-12-31,18,0
1,83195883,120,10781,27/12/2018 19:47,93,"Cloudesley Road, Angel",27/12/2018 19:45,339,"Risinghill Street, Angel",2018-12-27,19,3
2,83196070,120,2977,27/12/2018 20:11,339,"Risinghill Street, Angel",27/12/2018 20:09,234,"Liverpool Road (N1 Centre), Angel",2018-12-27,20,3
3,83197932,660,10802,28/12/2018 07:35,282,"Royal London Hospital, Whitechapel",28/12/2018 07:24,698,"Shoreditch Court, Haggerston",2018-12-28,7,4
4,83176351,1380,15749,26/12/2018 11:55,785,"Aquatic Centre, Queen Elizabeth Olympic Park",26/12/2018 11:32,783,"Monier Road, Hackney Wick",2018-12-26,11,2


In [26]:
# Convert the date column to datetime format
data_2019['Start Date Converted'] = pd.to_datetime(data_2019['Start Date Converted'])
data_2019.head()


,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,Start Date Converted,Hours,Day
0,83252102,720,2077,31/12/2018 19:05,272,"Baylis Road, Waterloo",31/12/2018 18:53,94,"Bricklayers Arms, Borough",2018-12-31,18,0
1,83195883,120,10781,27/12/2018 19:47,93,"Cloudesley Road, Angel",27/12/2018 19:45,339,"Risinghill Street, Angel",2018-12-27,19,3
2,83196070,120,2977,27/12/2018 20:11,339,"Risinghill Street, Angel",27/12/2018 20:09,234,"Liverpool Road (N1 Centre), Angel",2018-12-27,20,3
3,83197932,660,10802,28/12/2018 07:35,282,"Royal London Hospital, Whitechapel",28/12/2018 07:24,698,"Shoreditch Court, Haggerston",2018-12-28,7,4
4,83176351,1380,15749,26/12/2018 11:55,785,"Aquatic Centre, Queen Elizabeth Olympic Park",26/12/2018 11:32,783,"Monier Road, Hackney Wick",2018-12-26,11,2


In [29]:
# Filter the dataframe to only include data from 2019
# Remember, one of the merged csv files contain data between 26/12/2018 to 01/01/2019 - filtering will remove the 2018 data 
bike_data_2019 = data_2019[data_2019['Start Date Converted'].dt.year == 2019]
print(bike_data_2019.shape)

(10310063, 12)


### Exploratory Analysis of 2019 data 

In [41]:
# counting by start location
print(bike_data_2019['StartStation Name'].value_counts())

# counting by end location
print(bike_data_2019['EndStation Name'].value_counts())

Belgrove Street , King's Cross          84578
Waterloo Station 3, Waterloo            77998
Hyde Park Corner, Hyde Park             71834
Waterloo Station 1, Waterloo            65950
Albert Gate, Hyde Park                  55133
                                        ...  
Warwick Row, Westminster                  373
Blackfriars Station, St. Paul's           352
St Mary's Hospital, Westminster           119
Pop Up Dock 1                              57
Thessaly Road North, Wandsworth Road        7
Name: StartStation Name, Length: 801, dtype: int64


# Getting the bike station locations
##### TfL have a live "cycle hire updates" feed which lists information for each cycle hire station, updated once every minute or so. I don't utilise this live data - instead I just take the name, ID, lat/lon, and capacity for each bike station.

Code adopted from https://github.com/charlie1347/TfL_bikes/blob/master/TfL%20Bikes.ipynb


In [35]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd

site = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"

response = requests.get(site)
root = ET.fromstring(response.content)

id_list = [int(root[i][0].text) for i in range(0, len(root))]
name_list = [root[i][1].text for i in range(0, len(root))]
lat_list = [float(root[i][3].text) for i in range(0, len(root))]
lon_list = [float(root[i][4].text) for i in range(0, len(root))]
capacity_list = [int(root[i][12].text) for i in range(0, len(root))]

all_locs = pd.DataFrame(list(zip(name_list, id_list, lat_list, 
                                 lon_list, capacity_list)), columns = ["name","id","lat","lon","capacity"])

all_locs.to_csv('output/bike_point_locations_saved.csv', header=True, index=None)

print(all_locs.shape)


locations = all_locs.copy()

locations.head()

(798, 5)


,name,id,lat,lon,capacity
0,"River Street , Clerkenwell",1,51.529163,-0.109971,0
1,"Phillimore Gardens, Kensington",2,51.499607,-0.197574,0
2,"Christopher Street, Liverpool Street",3,51.521284,-0.084606,0
3,"St. Chad's Street, King's Cross",4,51.530059,-0.120974,0
4,"Sedding Street, Sloane Square",5,51.493130,-0.156876,0


### Creating an interative map of TfL bike docking stations

In [36]:
import folium 
# creating a folium map 

m = folium.Map(
    location=[51.5074, -0.1272], # start location as lat and lon
    tiles="CartoDB dark_matter", # adding a dark basemap
    zoom_start=12, # level of zoom
    prefer_canvas=True, # useful for changing the base map
)

In [37]:
# iterate through every row of the dataframe using the iterrows() function
for index, val in locations.iterrows():
    folium.CircleMarker(
        location=[val["lat"], val["lon"]],
        # styling the circles with different parameters 
        radius=5,
        popup= val["name"],
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    # adding all circles to the map
    ).add_to(m)


In [38]:
m

### Merging all bike data from 2019 and docking lcoations into one dataframe